In [21]:


import pandas as pd
import os
import mysql.connector

from decimal import getcontext
from enum import unique
from string import printable
from mysql.connector import Error
from tqdm import tqdm
from pprint import pprint


In [ ]:

# %pip install Unidecode nltk 

In [22]:


def get_conn():
    return mysql.connector.connect(
            host='192.168.0.217',
            database='propiedades',
            user='root',
            password='propiedades')

def test_connection():
    try:
        connection = get_conn()
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


connection = get_conn()
cursor = connection.cursor()


def read_all(query, cursor=cursor):
    cursor.execute(query)
    return cursor.fetchall()

def print_all(query, cursor=cursor):
    records = read_all(query, cursor)
    pprint(records)


In [23]:


# unique_states = [('Tamaulipas',), ('México',), ('Sinaloa',), ('Campeche',), ('DF / CDMX',), ('Coahuila de Zaragoza',), ('Chihuahua',), ('Durango',), ('Jalisco',), ('Colima',), ('Veracruz de Ignacio de la Llave',), ('Guanajuato',), ('Puebla',), ('Sonora',), ('Oaxaca',), (None,), ('Chiapas',), ('Quintana Roo',), ('Michoacán de Ocampo',), ('Baja California Sur',), ('Aguascalientes',), ('Baja California',), ('Yucatán',), ('Nayarit',), ('Guerrero',), ('Morelos',), ('Nuevo León',), ('Querétaro',), ('Hidalgo',), ('Tlaxcala',), ('Tabasco',), ('San Luis Potosí',), ('Zacatecas',), ('Distrito Federal',)]
unique_states = read_all("SELECT DISTINCT state FROM sepomex", cursor)
result = read_all('DESC sepomex', cursor)
fields = [t[0] for t in result]


In [106]:
unique_states.sort()

In [ ]:
import nltk
import unidecode
import operator

from nltk.corpus import stopwords
from difflib import SequenceMatcher


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


nltk.download("stopwords")
stopwords = stopwords.words('spanish')

In [25]:

def download_csv_from_mysql(cursor, unique_states, fields):
    n = len(unique_states)
    pbar = tqdm(enumerate(unique_states), total=n)
    for i, state in pbar:
        state = state[0]
        query = f"SELECT {', '.join(fields)} FROM sepomex WHERE state = '{state}'"
        res = read_all(query, cursor)
        df = pd.DataFrame(data=res)
        df.columns = fields
        df.to_csv(f"csv/{i}.csv", index=False)
        pbar.set_description(state)


download_csv_from_mysql(cursor, unique_states, fields)

Zacatecas: 100%|██████████| 32/32 [15:18<00:00, 28.69s/it]                      


In [ ]:

# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [115]:

def get_df_from_sources(partition_fname):
    cols = ['id', 'colony']
    df_sql = pd.read_csv(partition_fname)
    values_sql = df_sql[~df_sql['colony'].isnull()][cols].values

    state_label = df_sql.loc[0,'state']

    df_web = pd.read_csv("./CPdescarga.csv", sep="|", encoding='utf8')
    subset = df_web[df_web["d_estado"] == state_label]
    cols = ['d_codigo', 'd_asenta']
    values_web = subset[cols].values

    return values_sql, values_web


In [108]:

def clean_str(s):
    s = s.lower()
    tokens = s.split(' ')
    new_s = [t for t in tokens if not t in stopwords]
    s = ' '.join(new_s)
    return unidecode.unidecode(s)




In [109]:

def get_data(values_sql, values_web):
    threshold = 0.8
    data = []  # idsql, idweb, new|update
    pbar = tqdm(values_sql, total=len(values_sql))
    for i, col_sql in pbar:
        sims = [(j, similar(col_sql, col_web)) for j, col_web in values_web]
        sims.sort(key=operator.itemgetter(1), reverse=True)
        j, sim= sims[0]
        if sim == 0.0:  
            data.append([i, col_sql, "insert", sims[:5]])
        elif 1 > sim and sim > threshold:
            data.append([i, col_sql, "update", sims[:5]])



100%|██████████| 1715/1715 [02:43<00:00, 10.51it/s]


In [118]:

fnames = os.listdir('csv')
n = len(fnames)
for k, fname in enumerate(fnames):
    print("%d/%d in progress" % (k, n))
    values_sql, values_web = get_df_from_sources("csv/" + fname)
    values_sql = [(i, clean_str(s)) for i, s in values_sql]
    values_web = [(i, clean_str(s)) for i, s in values_web]
    data = get_data(values_sql, values_web)
    df = pd.DataFrame(data, columns=['sql_id', 'colony_sql_name', 'action_string', 'knn_list'])
    df.to_csv("result/" + fname, index=False)

0/32 in progress


100%|██████████| 1299/1299 [01:38<00:00, 13.21it/s]


1/32 in progress


100%|██████████| 2228/2228 [05:50<00:00,  6.36it/s]


2/32 in progress


 14%|█▎        | 1204/8853 [12:19<1:26:32,  1.47it/s]

In [ ]:

print(similar("angel", "angel"))
print(similar("angel", "a n g e l"))
print(similar("angel", "Angel"))
print(similar("angel", "angelito"))
print(similar("angel", "arcangel"))
print(similar("angel", "angelopolis"))
print(similar("angel", "carburador"))



0.13333333333333333